In [ ]:
!pip install pandas numpy matplotlib scikit-learn

In [ ]:
"""
reproduce_paper_models.py

Implements paper-like experiments:
 - NAR-style models: one scalar-output MLP per horizon (h = 1..H)
 - LSTM models: one scalar-output LSTM per horizon (h = 1..H)

Notes:
 - The paper used Levenberg-Marquardt + Bayesian regularization for NAR (MATLAB).
   This exact optimizer is not available in TF/Keras. We use Adam for NAR here.
 - LSTM uses AdaGrad (we use Keras Adagrad).
 - Training set sizes: NAR -> 5000, LSTM -> 100000 (defaults, can be changed).
 - WARNING: training 120 separate models can be expensive. You can restrict horizons
   using H_MAX to experiment quickly.
"""

import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, LSTM, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# ===========================
# USER PARAMETERS
# ===========================
DATA_PATH = "/content/household_power_consumption.txt"  # change if needed
INPUT_COL = 'Global_active_power'
PAST_STEPS = 30
HORIZON = 120  # paper uses 120
MAX_SAMPLES = None  # set to None to use all windows or e.g. 50000 to limit
TEST_RATIO = 0.15
VAL_RATIO = 0.15
RANDOM_SEED = 42
BATCH_SIZE = 128
EPOCHS_NAR = 100  # you can reduce for quick runs
EPOCHS_LSTM = 50
NAR_TRAIN_SAMPLES = 5000     # paper used ~5k for NAR
LSTM_TRAIN_SAMPLES = 100000  # paper used ~100k for LSTM
MODEL_DIR = "paper_models"   # saved models folder
H_MAX = HORIZON  # set lower (e.g., 12) to speed up testing

os.makedirs(MODEL_DIR, exist_ok=True)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# ===========================
# 0) Load & preprocess
# ===========================
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Data file not found: {DATA_PATH}")

print("Loading data...")
df = pd.read_csv(
    DATA_PATH,
    sep=';',
    parse_dates={'datetime': ['Date', 'Time']},
    infer_datetime_format=True,
    low_memory=False,
    na_values=['?','']
)
df.sort_values('datetime', inplace=True)
df.set_index('datetime', inplace=True)

# convert target
df[INPUT_COL] = pd.to_numeric(df[INPUT_COL], errors='coerce')
# resample to 1-minute
df = df.resample('1T').asfreq()

# interpolate short gaps (paper used limit=60)
df[INPUT_COL] = df[INPUT_COL].interpolate(method='time', limit=60)
df = df.dropna(subset=[INPUT_COL])  # drop any remaining NaNs

series = df[[INPUT_COL]].astype('float32')
T = len(series)
print("Timesteps after cleaning:", T)

# ===========================
# 1) Build sliding windows indices
# ===========================
total_steps = len(series)
max_start_idx = total_steps - (PAST_STEPS + HORIZON) + 1
if max_start_idx <= 0:
    raise ValueError("Not enough data for requested window sizes.")

n_windows = max_start_idx
test_windows = int(np.ceil(TEST_RATIO * n_windows))
val_windows = int(np.ceil(VAL_RATIO * n_windows))
train_windows = n_windows - val_windows - test_windows
print(f"Windows: total={n_windows}, train={train_windows}, val={val_windows}, test={test_windows}")

start_indices = np.arange(0, n_windows, dtype=np.int64)
if MAX_SAMPLES is not None:
    start_indices = start_indices[:MAX_SAMPLES]
    n_windows = len(start_indices)
    test_windows = int(np.ceil(TEST_RATIO * n_windows))
    val_windows = int(np.ceil(VAL_RATIO * n_windows))
    train_windows = n_windows - val_windows - test_windows
    print(f"Limited windows -> total={n_windows}, train={train_windows}, val={val_windows}, test={test_windows}")

train_starts = start_indices[:train_windows]
val_starts = start_indices[train_windows:train_windows + val_windows]
test_starts = start_indices[train_windows + val_windows: train_windows + val_windows + test_windows]

# ===========================
# 2) Fit scaler on training inputs only
# ===========================
scaler = MinMaxScaler(feature_range=(0,1))
# Fit on training inputs region (up to last input index present in training)
last_input_idx = train_starts[-1] + PAST_STEPS - 1
fit_slice = series.iloc[: last_input_idx + 1]
scaler.fit(fit_slice.values)
print("Scaler fit on training region:", fit_slice.shape)

# helper: build X (past) and scalar y for a given horizon h (1-based)
def build_xy_scalar_h(series_values, starts, past_steps, h, scaler):
    """
    returns X shape (N, past_steps, 1), y shape (N,)
    h: forecast horizon in steps ahead (1..HORIZON)
    """
    n = len(starts)
    X = np.zeros((n, past_steps, series_values.shape[1]), dtype=np.float32)
    y = np.zeros((n,), dtype=np.float32)
    for i, s in enumerate(starts):
        in_start = s
        in_end = s + past_steps
        out_idx = in_end + (h - 1)  # zero-based index of the scalar target
        window_in = series_values[in_start:in_end]          # shape (past_steps, 1)
        target = series_values[out_idx, 0]                  # scalar
        X[i] = scaler.transform(window_in)
        y[i] = scaler.transform(np.array([[target]]))[0,0]
    return X, y

series_values = series.values  # (T,1)

# ===========================
# 3) Helper to create and train NAR model for one horizon
# ===========================
def make_nar_model(past_steps):
    # Single hidden layer, 40 neurons, sigmoid activation, linear output
    model = Sequential([
        Input(shape=(past_steps,1)),
        Flatten(),
        Dense(40, activation='sigmoid'),
        Dense(1, activation='linear')
    ])
    # NOTE: paper used Levenberg-Marquardt + Bayesian reg (MATLAB). Not available in Keras.
    # We use Adam here for practicality. Replace optimizer below if desired.
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='mae', metrics=['mae'])
    return model

# ===========================
# 4) Helper to create and train LSTM model for one horizon
# ===========================
def make_lstm_model(past_steps):
    # LSTM with 50 units and softsign activation, linear output
    model = Sequential([
        Input(shape=(past_steps,1)),
        LSTM(50, activation='softsign'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=1e-2), loss='mae', metrics=['mae'])
    return model

# ===========================
# 5) Training loop: NAR models (one per horizon)
# ===========================
# To save time you can restrict H_MAX (e.g., 12 or 24). Set H_MAX = HORIZON for full run.
H_MAX = min(H_MAX, HORIZON)
print(f"Training NAR models for horizons 1..{H_MAX} (paper used 120)")
nar_results = {}

for h in range(1, H_MAX + 1):
    print(f"\n=== NAR: Horizon h={h} ===")
    # build datasets (train/val/test) for this horizon
    X_tr, y_tr = build_xy_scalar_h(series_values, train_starts, PAST_STEPS, h, scaler)
    X_val, y_val = build_xy_scalar_h(series_values, val_starts, PAST_STEPS, h, scaler)
    X_te, y_te = build_xy_scalar_h(series_values, test_starts, PAST_STEPS, h, scaler)

    # Optionally limit training samples to match paper's approx regime
    if NAR_TRAIN_SAMPLES is not None and NAR_TRAIN_SAMPLES < X_tr.shape[0]:
        X_tr_use = X_tr[:NAR_TRAIN_SAMPLES]
        y_tr_use = y_tr[:NAR_TRAIN_SAMPLES]
    else:
        X_tr_use = X_tr; y_tr_use = y_tr

    model = make_nar_model(PAST_STEPS)
    fname = os.path.join(MODEL_DIR, f"nar_h{h:03d}.h5")
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1),
        ModelCheckpoint(fname, monitor='val_loss', save_best_only=True, verbose=0)
    ]
    history = model.fit(
        X_tr_use, y_tr_use,
        validation_data=(X_val, y_val),
        epochs=EPOCHS_NAR,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        shuffle=False,
        verbose=2
    )

    # evaluate on test set
    y_pred = model.predict(X_te).reshape(-1)
    # inverse scale
    y_te_inv = scaler.inverse_transform(y_te.reshape(-1,1)).reshape(-1)
    y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1,1)).reshape(-1)
    mae = mean_absolute_error(y_te_inv, y_pred_inv)
    print(f"NAR h={h} test MAE: {mae:.4f} (units same as original series)")
    nar_results[h] = {'model_path': fname, 'test_mae': float(mae)}

# Save summary
import json
with open(os.path.join(MODEL_DIR, "nar_summary.json"), "w") as f:
    json.dump(nar_results, f, indent=2)

# ===========================
# 6) Training loop: LSTM models (one per horizon)
# ===========================
# LSTM training uses larger training set by default. Be mindful of time.
H_MAX_LSTM = H_MAX  # you can set separately
print(f"\nTraining LSTM models for horizons 1..{H_MAX_LSTM}")
lstm_results = {}

for h in range(1, H_MAX_LSTM + 1):
    print(f"\n=== LSTM: Horizon h={h} ===")
    X_tr, y_tr = build_xy_scalar_h(series_values, train_starts, PAST_STEPS, h, scaler)
    X_val, y_val = build_xy_scalar_h(series_values, val_starts, PAST_STEPS, h, scaler)
    X_te, y_te = build_xy_scalar_h(series_values, test_starts, PAST_STEPS, h, scaler)

    # limit training samples to match paper guidance for LSTM
    if LSTM_TRAIN_SAMPLES is not None and LSTM_TRAIN_SAMPLES < X_tr.shape[0]:
        X_tr_use = X_tr[:LSTM_TRAIN_SAMPLES]
        y_tr_use = y_tr[:LSTM_TRAIN_SAMPLES]
    else:
        X_tr_use = X_tr; y_tr_use = y_tr

    model = make_lstm_model(PAST_STEPS)
    fname = os.path.join(MODEL_DIR, f"lstm_h{h:03d}.h5")
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1),
        ModelCheckpoint(fname, monitor='val_loss', save_best_only=True, verbose=0)
    ]

    history = model.fit(
        X_tr_use, y_tr_use,
        validation_data=(X_val, y_val),
        epochs=EPOCHS_LSTM,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        shuffle=False,
        verbose=2
    )

    y_pred = model.predict(X_te).reshape(-1)
    y_te_inv = scaler.inverse_transform(y_te.reshape(-1,1)).reshape(-1)
    y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1,1)).reshape(-1)
    mae = mean_absolute_error(y_te_inv, y_pred_inv)
    print(f"LSTM h={h} test MAE: {mae:.4f}")
    lstm_results[h] = {'model_path': fname, 'test_mae': float(mae)}

with open(os.path.join(MODEL_DIR, "lstm_summary.json"), "w") as f:
    json.dump(lstm_results, f, indent=2)

print("Done. Models & summaries saved in:", MODEL_DIR)


Loading data...


/tmp/ipython-input-1652253647.py:57: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
/tmp/ipython-input-1652253647.py:57: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(
/tmp/ipython-input-1652253647.py:57: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(
/tmp/ipython-input-1652253647.py:71: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample('1T').asfreq()


Timesteps after cleaning: 2050108
Windows: total=2049959, train=1434971, val=307494, test=307494
Scaler fit on training region: (1435000, 1)
Training NAR models for horizons 1..120 (paper used 120)

=== NAR: Horizon h=1 ===


In [ ]:
DATA_PATH = "/content/household_power_consumption.txt"
print("Dataset found:", DATA_PATH)

# ============================
# Imports
# ============================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# ============================
# Parameters
# ============================
TXT_FILENAME = DATA_PATH
INPUT_COL = 'Global_active_power'
PAST_STEPS = 30
FUTURE_STEPS = 120
TEST_RATIO = 0.15
VAL_RATIO = 0.15
BATCH_SIZE = 128
EPOCHS = 50
RANDOM_SEED = 42
MODEL_SAVE_PATH = 'best_simple_mlp.h5'

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# ============================
# Load & preprocess data
# ============================
df = pd.read_csv(
    TXT_FILENAME,
    sep=';',
    parse_dates={'datetime': ['Date', 'Time']},
    na_values=['?', ''],
    low_memory=False
)

df.sort_values('datetime', inplace=True)
df.set_index('datetime', inplace=True)

df[INPUT_COL] = pd.to_numeric(df[INPUT_COL], errors='coerce')
df = df.resample('1T').asfreq()
df[INPUT_COL] = df[INPUT_COL].interpolate(method='time', limit=60)
df = df.dropna(subset=[INPUT_COL])

series = df[[INPUT_COL]].astype('float32')
series_values = series.values

# ============================
# Sliding windows
# ============================
total_steps = len(series)
n_windows = total_steps - (PAST_STEPS + FUTURE_STEPS) + 1

test_windows = int(np.ceil(TEST_RATIO * n_windows))
val_windows = int(np.ceil(VAL_RATIO * n_windows))
train_windows = n_windows - test_windows - val_windows

starts = np.arange(n_windows)
train_starts = starts[:train_windows]
val_starts = starts[train_windows:train_windows + val_windows]
test_starts = starts[train_windows + val_windows:]

# ============================
# Scaling (train-only)
# ============================
scaler = MinMaxScaler()
last_train_idx = train_starts[-1] + PAST_STEPS
scaler.fit(series_values[:last_train_idx])

def build_xy(starts):
    X = np.zeros((len(starts), PAST_STEPS, 1))
    Y = np.zeros((len(starts), FUTURE_STEPS))
    for i, s in enumerate(starts):
        X[i] = scaler.transform(series_values[s:s+PAST_STEPS])
        Y[i] = scaler.transform(series_values[s+PAST_STEPS:s+PAST_STEPS+FUTURE_STEPS]).reshape(-1)
    return X, Y

X_train, Y_train = build_xy(train_starts)
X_val, Y_val = build_xy(val_starts)
X_test, Y_test = build_xy(test_starts)

# ============================
# Build MLP
# ============================
tf.keras.backend.clear_session()

model = Sequential([
    Input(shape=(PAST_STEPS, 1)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(FUTURE_STEPS, activation='linear')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='mae'
)

model.summary()

# ============================
# Train
# ============================
callbacks = [
    EarlyStopping(patience=6, restore_best_weights=True),
    ModelCheckpoint(MODEL_SAVE_PATH, save_best_only=True)
]

model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=False,
    callbacks=callbacks
)

# ============================
# Predict & inverse scale
# ============================
Y_pred = model.predict(X_test)

Y_test_inv = scaler.inverse_transform(Y_test.reshape(-1,1)).reshape(-1, FUTURE_STEPS)
Y_pred_inv = scaler.inverse_transform(Y_pred.reshape(-1,1)).reshape(-1, FUTURE_STEPS)

# ============================
# Metrics per horizon
# ============================
mae_scores = np.zeros(FUTURE_STEPS)
mse_scores = np.zeros(FUTURE_STEPS)
r2_scores  = np.zeros(FUTURE_STEPS)

for i in range(FUTURE_STEPS):
    mae_scores[i] = mean_absolute_error(Y_test_inv[:, i], Y_pred_inv[:, i])
    mse_scores[i] = mean_squared_error(Y_test_inv[:, i], Y_pred_inv[:, i])
    r2_scores[i]  = r2_score(Y_test_inv[:, i], Y_pred_inv[:, i])

error_variance = np.var(Y_test_inv - Y_pred_inv, axis=0)

# ============================
# PLOTS (MATCH FRIEND)
# ============================

# MAE
plt.figure(figsize=(12,6))
plt.plot(range(1, FUTURE_STEPS+1), mae_scores, label='MLP - MAE')
plt.xlabel('Forecast Span [minutes]')
plt.ylabel('Mean Absolute Error (MAE)')
plt.title('MAE for Multi-Step Forecasting (MLP)')
plt.legend()
plt.grid(False)
plt.show()

# MSE
plt.figure(figsize=(12,6))
plt.plot(range(1, FUTURE_STEPS+1), mse_scores, label='MLP - MSE')
plt.xlabel('Forecast Span [minutes]')
plt.ylabel('Mean Squared Error (MSE)')
plt.title('MSE for Multi-Step Forecasting (MLP)')
plt.legend()
plt.grid(False)
plt.show()

# R²
plt.figure(figsize=(12,6))
plt.plot(range(1, FUTURE_STEPS+1), r2_scores, label='MLP - R²')
plt.xlabel('Forecast Span [minutes]')
plt.ylabel('R² Score')
plt.title('R² for Multi-Step Forecasting (MLP)')
plt.legend()
plt.grid(False)
plt.show()

# Error Variance
plt.figure(figsize=(12,6))
plt.plot(range(1, FUTURE_STEPS+1),
         error_variance,
         label='MLP - Error Variance',
         marker='o',
         markersize=4)

plt.xlabel('Forecast Span [minutes]')
plt.ylabel('Error Variance')
plt.title('Error Variance for Multi-Step Forecasting (MLP)')
plt.legend()
plt.grid(False)
plt.show()

# ============================
# Save
# ============================
np.savez(
    'simple_mlp_results_full.npz',
    mae=mae_scores,
    mse=mse_scores,
    r2=r2_scores,
    error_variance=error_variance,
    Y_test=Y_test_inv,
    Y_pred=Y_pred_inv
)

print("✅ DONE — All graphs generated (MAE, MSE, R², Error Variance)")
print("Model saved to:", MODEL_SAVE_PATH)


In [ ]:
# ---------------------------
# LSTM model for multi-step forecasting
# Re-uses arrays: X_train, Y_train, X_val, Y_val, X_test, Y_test, scaler
# and constants: PAST_STEPS, FUTURE_STEPS, BATCH_SIZE, EPOCHS, RANDOM_SEED
# ---------------------------

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

LSTM_MODEL_SAVE_PATH = 'best_lstm.h5'

# reproducibility (already set earlier, but safe to set again)
import tensorflow as tf
tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# clear session to avoid clutter
tf.keras.backend.clear_session()

# Build LSTM model
# Paper config: one hidden layer with 50 memory cells (units), softsign activation, linear output.
lstm_model = Sequential([
    Input(shape=(PAST_STEPS, n_features)),         # n_features should be 1 for your dataset
    LSTM(50, activation='softsign'),              # 50 memory cells as in paper
    # optional dropout to reduce overfitting (tune or remove)
    Dropout(0.1),
    Dense(FUTURE_STEPS, activation='linear')      # predict all FUTURE_STEPS in one shot
])

# Compile with Adagrad optimizer (paper used ADAGRAD). Use MAE loss to match MLP experiment.
lstm_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=1e-2),
                   loss='mae',
                   metrics=['mae'])

lstm_model.summary()

# Callbacks: early stopping and checkpointing (save best by val_loss)
callbacks = [
    EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1),
    ModelCheckpoint(LSTM_MODEL_SAVE_PATH, monitor='val_loss', save_best_only=True, verbose=1)
]

# Fit model
history_lstm = lstm_model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    shuffle=False
)

# ---------------------------
# Evaluation - MAE per horizon
# ---------------------------
print("Predicting on test set (LSTM)...")
Y_pred_test_lstm = lstm_model.predict(X_test)  # shape (N, FUTURE_STEPS)

# Inverse transform outputs back to original scale
Y_test_inv_lstm = scaler.inverse_transform(Y_test.reshape(-1, 1)).reshape(-1, FUTURE_STEPS)
Y_pred_inv_lstm = scaler.inverse_transform(Y_pred_test_lstm.reshape(-1, 1)).reshape(-1, FUTURE_STEPS)

# compute MAE per horizon
from sklearn.metrics import mean_absolute_error
mae_per_horizon_lstm = []
for h in range(FUTURE_STEPS):
    mae_h = mean_absolute_error(Y_test_inv_lstm[:, h], Y_pred_inv_lstm[:, h])
    mae_per_horizon_lstm.append(mae_h)

overall_mae_lstm = mean_absolute_error(Y_test_inv_lstm.reshape(-1), Y_pred_inv_lstm.reshape(-1))
print(f"LSTM overall test MAE (all horizons): {overall_mae_lstm:.4f} kW")

# Plot MAE vs horizon (same plotting style as MLP)
plt.figure(figsize=(10,5))
plt.plot(np.arange(1, FUTURE_STEPS + 1), mae_per_horizon_lstm, marker='o', markersize=3)
plt.xlabel('Minutes ahead')
plt.ylabel('MAE (kW)')
plt.title('MAE vs Forecast Horizon (LSTM)')
plt.grid(True)
plt.show()

# Plot sample predictions (like MLP)
n_samples_plot = 3
fig, axes = plt.subplots(n_samples_plot, 1, figsize=(10, 3*n_samples_plot))
for i in range(n_samples_plot):
    ax = axes[i]
    idx = i if i < X_test.shape[0] else 0
    input_window = scaler.inverse_transform(X_test[idx].reshape(-1, 1)).reshape(-1)
    actual_future = Y_test_inv_lstm[idx]
    pred_future = Y_pred_inv_lstm[idx]
    t_input = np.arange(-PAST_STEPS, 0)
    t_future = np.arange(0, FUTURE_STEPS)
    ax.plot(t_input, input_window, label='Input (past 30 min)', marker='o')
    ax.plot(t_future, actual_future, label='Actual future (next 120 min)')
    ax.plot(t_future, pred_future, label='Predicted future (LSTM)')
    ax.axvline(-0.5, color='k', linestyle='--')
    ax.set_xlabel('Minutes (relative)')
    ax.set_ylabel('Global_active_power (kW)')
    ax.legend()
    ax.grid(True)
plt.tight_layout()
plt.show()

# Save results
np.savez('lstm_results.npz',
         mae_per_horizon=np.array(mae_per_horizon_lstm),
         Y_test=Y_test_inv_lstm,
         Y_pred=Y_pred_inv_lstm)

print("Done. LSTM model saved to:", LSTM_MODEL_SAVE_PATH)
print("Results saved to lstm_results.npz")

# ----------------------------------------------
# EXTRA: Three plots for report (MAE, MSE, VAR)
# ----------------------------------------------

from sklearn.metrics import mean_absolute_error, mean_squared_error

# ============= 1) MAE per forecast horizon =============
mae_per_horizon = []
for h in range(FUTURE_STEPS):
    mae = mean_absolute_error(Y_test_inv_lstm[:, h], Y_pred_inv_lstm[:, h])
    mae_per_horizon.append(mae)

plt.figure(figsize=(10,5))
plt.plot(range(1, FUTURE_STEPS+1), mae_per_horizon, marker='o', markersize=3)
plt.xlabel("Forecast Span [minutes]")
plt.ylabel("MAE (kWh)")
plt.title("Mean Absolute Error (MAE) vs Forecast Span (LSTM)")
plt.grid(True)
plt.show()


# ============= 2) MSE per forecast horizon =============
mse_per_horizon = []
for h in range(FUTURE_STEPS):
    mse = mean_squared_error(Y_test_inv_lstm[:, h], Y_pred_inv_lstm[:, h])
    mse_per_horizon.append(mse)

plt.figure(figsize=(10,5))
plt.plot(range(1, FUTURE_STEPS+1), mse_per_horizon, marker='o', markersize=3)
plt.xlabel("Forecast Span [minutes]")
plt.ylabel("MSE (kWh)")
plt.title("Mean Squared Error (MSE) vs Forecast Span (LSTM)")
plt.grid(True)
plt.show()


# ============= 3) Error Variance per horizon =============
error_variance = []
for h in range(FUTURE_STEPS):
    errors = Y_test_inv_lstm[:, h] - Y_pred_inv_lstm[:, h]
    error_variance.append(np.var(errors))

plt.figure(figsize=(10,5))
plt.plot(range(1, FUTURE_STEPS+1), error_variance, marker='o', markersize=3)
plt.xlabel("Forecast Span [minutes]")
plt.ylabel("Error Variance")
plt.title("Error Variance vs Forecast Span (LSTM)")
plt.grid(True)
plt.show()

print("Generated MAE, MSE, and Error Variance plots for report.")


NameError: name 'n_features' is not defined